In [ ]:
!wget https://github.com/pauldebayan/RockPaperScissor-Game/raw/refs/heads/main/rockPaperScissor_dataset.zip
!wget https://raw.githubusercontent.com/pauldebayan/RockPaperScissor-Game/refs/heads/main/labels.csv
!unzip rockPaperScissor_dataset.zip
!rm -rf rockPaperScissor_dataset.zip

In [ ]:
!unzip rockPaperScissor_dataset.zip
!rm -rf rockPaperScissor_dataset.zip

!unzip 2.zip
!rm -rf 2.zip

In [ ]:
from torchvision.io import read_image
import matplotlib.pyplot as plt
import torchvision
import torch
import torch.nn as nn
import os
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
img = read_image('./rockPaperScissor_dataset/rock1.jpg')

print(img.shape[0])

In [ ]:
# for i in range(2500):
#     name = './rockPaperScissor_dataset/scissor'+str(i+1)+'.jpg'
#     img = read_image(name)
#     if img.shape[0] != 3 or img.shape[1] != 300 or img.shape[2] != 300:
#         print(name)

In [ ]:
class RockPaperScissorDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

In [ ]:
dataset = RockPaperScissorDataset(img_dir = 'rockPaperScissor_dataset',
                             annotations_file = 'labels.csv',
                             transform = transform)

In [ ]:
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

In [ ]:
train_dataloader = DataLoader(train_set, batch_size=10, shuffle=True)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=1, shuffle=True)

In [ ]:
class RockPaperScissorCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 3, stride = 1)
        self.conv2 = nn.Conv2d(10, 8, 3, stride = 1)
        self.conv3 = nn.Conv2d(8, 3, 3, stride = 1)

        self.pool = nn.MaxPool2d(2, 2)

        self.activ = nn.LeakyReLU()

        self.dropout = nn.Dropout(0.2)

        self.linear_stack = nn.Sequential(
            nn.Linear(3675, 20),
            nn.LeakyReLU(),
            nn.Linear(20, 10),
            nn.LeakyReLU(),
            nn.Linear(10, 3),
        )

    def forward(self, x):

        x = self.pool(self.activ(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(self.activ(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(self.activ(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x = self.linear_stack(x)

        return x

In [ ]:
model = RockPaperScissorCNN().to(device)
print(model)

In [ ]:
print(img.shape)
model(img.float().to(device))

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
img = train_features[0]
label = train_labels[0]

print(img.shape)

In [ ]:
learning_rate = 0.0001
batch_size = 5
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop():

    size = len(train_dataloader)
    accumulate_loss = 0

    for img, label in train_dataloader:

        img = img.float().to(device)
        label = label.to(device)

        pred = model(img)

        loss = loss_fn(pred, label)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        accumulate_loss += loss.item()

    return accumulate_loss/size


def valid_loop():
    size = len(valid_dataloader)
    accumulate_loss = 0

    for img, label in valid_dataloader:

        img = img.float().to(device)
        label = label.to(device)

        pred = model(img)
        loss = loss_fn(pred, label)

        accumulate_loss += loss.item()


    return accumulate_loss/size



In [ ]:
x = [x+1 for x in range(epochs)]
trainLossArr = []
validLossArr = []

for i in range(epochs):

    # Train
    train_loss = train_loop()
    trainLossArr.append(train_loss)

    # Valid
    valid_loss = train_loop()
    validLossArr.append(valid_loss)

    print(f"Epoch {i+1} - Training Loss: {train_loss}, Validation Loss: {valid_loss}")


plt.plot(x, trainLossArr, 'b', label="Train")
plt.plot(x, validLossArr, 'g', label="Valid")
plt.legend(loc="upper right")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [83]:
torch.save(model.state_dict(), 'model.pt')

In [84]:
state_dict = torch.load('model.pt', map_location=device, weights_only=True)
model.load_state_dict(state_dict)
model.eval()

RockPaperScissorCNN(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(10, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(8, 3, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (activ): LeakyReLU(negative_slope=0.01)
  (dropout): Dropout(p=0.2, inplace=False)
  (linear_stack): Sequential(
    (0): Linear(in_features=3675, out_features=20, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=10, out_features=3, bias=True)
  )
)

In [85]:
correct = 0
for img, label in test_dataloader:
    img = img.float().to(device)
    label = label.to(device)
    pred = model(img)

    if torch.argmax(pred) == label.squeeze():
        correct += 1
accuracy = correct/len(test_dataloader)*100
print(f"Accuracy: {accuracy}")


Accuracy: 93.59145527369826


In [86]:
!pip install onnx

In [87]:
import onnx

img = torch.randn(1, 3, 300, 300).to(device)

torch.onnx.export(model, img, "model.onnx", input_names=['input'])

============ Diagnostic Run torch.onnx.export version 2.0.0.post304 ============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
from google.colab import files
files.download('model.onnx')